In [1]:
from keras.applications import inception_v3
from keras import backend as K

K.set_learning_phase(0)

model = inception_v3.InceptionV3(weights='imagenet',
                                 include_top=False)

Using TensorFlow backend.
/Users/johnwilson/anaconda/envs/ztdl/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}

In [3]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output

    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

In [4]:
dream = model.input

grads = K.gradients(loss, dream)[0]

grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

In [19]:
import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)

def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [22]:
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 1.,
    'mixed4': 2.,
    'mixed5': 1.5,
}
import numpy as np
step = 0.01
num_ocatve = 3
octave_scale = 1.4
iterations = 20
max_loss = 10.
base_image_path = "/Users/johnwilson/Desktop/starry_night.png"
img = preprocess_image(base_image_path)
img = resize_img(img,(512,512))
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1,num_ocatve):
    shape = tuple([int(dim/ (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)
successive_shapes = successive_shapes[::-1]    
original_img = np.copy(img)
shrunk_original_img = resize_img(img,successive_shapes[0])
for shape in successive_shapes:
    print('Processing image shape',shape)
    img = resize_img(img,shape)
    img = gradient_ascent(img,iterations = iterations,step=step,max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img,shape)
    same_size_original = resize_img(original_img,shape)
    lost_detail = same_size_original-upscaled_shrunk_original_img
    img+=lost_detail
    shrunk_original_img =resize_img(original_img,shape)
    save_img(img,fname = 'dream_at_scale_'+str(shape) + '.png')
save_img(img,fname='final_dream.png')    

Processing image shape (261, 261)
...Loss value at 0 : 1.4971873
...Loss value at 1 : 2.2473972
...Loss value at 2 : 2.904879
...Loss value at 3 : 3.4913092
...Loss value at 4 : 4.107569
...Loss value at 5 : 4.744383
...Loss value at 6 : 5.359408
...Loss value at 7 : 5.9777737
...Loss value at 8 : 6.519592
...Loss value at 9 : 7.0759463
...Loss value at 10 : 7.6029367
...Loss value at 11 : 8.079557
...Loss value at 12 : 8.581804
...Loss value at 13 : 9.054133
...Loss value at 14 : 9.516419
...Loss value at 15 : 9.928343


/Users/johnwilson/anaconda/envs/ztdl/lib/python3.5/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


Processing image shape (365, 365)
...Loss value at 0 : 2.5275903
...Loss value at 1 : 3.701915
...Loss value at 2 : 4.7458315
...Loss value at 3 : 5.733871
...Loss value at 4 : 6.5648246
...Loss value at 5 : 7.3750353
...Loss value at 6 : 8.099675
...Loss value at 7 : 8.820207
...Loss value at 8 : 9.483015
Processing image shape (512, 512)
...Loss value at 0 : 2.666716
...Loss value at 1 : 3.8590834
...Loss value at 2 : 4.8839736
...Loss value at 3 : 5.854912
...Loss value at 4 : 6.7803617
...Loss value at 5 : 7.6688523
...Loss value at 6 : 8.5126915
...Loss value at 7 : 9.314524
